# Logistic Regression

In [1]:


from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score, f1_score, recall_score

def input_mlflow(cv_r_v,increment_number,X_train, y_train,pipeline,run_name,experiment_id,X_test,y_test):
    

    for wi in range(len(cv_r_v)):
        
        #====================skema===================
        cols_spec = []
        data_map = {
                'int64': 'integer',
                'float64': 'double',
                'bool': 'boolean',
                'str': 'string',
                'object': 'string',
                "date": 'datetime'
            }

        for name, dtype in X_train.dtypes.to_dict().items():
            cols_spec.append(ColSpec(name=name, type=data_map[str(dtype)]))
        input_schema = Schema(inputs=cols_spec)
        output_schema = Schema([ColSpec(name="label", type="string")])
        #parameter = ParamSpec(name="model_name", dtype="string", default="model1")
        #param_schema = ParamSchema(params=[parameter])
        model_signature = ModelSignature(inputs=input_schema, outputs=output_schema)#, params=param_schema)
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        model_signature = infer_signature(X_train, y_train)#, params={"model_name": "model1"})
        #print("MODEL SIGNATURE")
        #print(model_signature.to_dict())

        
        
        
        
        #====================buat run baru===================
        run_name_with_increment = f"{run_name}__{increment_number}"
            # Membuka run MLflow
        with mlflow.start_run(run_name=run_name_with_increment, experiment_id=experiment_id) as run:
            # Mendapatkan run_id
            run_id = mlflow.active_run().info.run_id
        
        print(f"mlruns/{experiment_id}/{run_id}/artifacts/grid_search__{increment_number}")
        increment_number=increment_number+1
        
        
        #====================buat parameter model dan metric===================

        # Definisikan grid parameter untuk dicari
        param_grid = cv_r_v[wi]
        
        #====================model===================

        # Inisialisasi GridSearchCV
        grid_search = GridSearchCV(pipeline, param_grid, cv=5,  scoring='accuracy')

        # Lakukan pencarian grid
        grid_search.fit(X_train, y_train)
        
            
        #====================parameter===================
        pipe=grid_search.best_estimator_
        
        
        first_step_name = list(pipe.named_steps.keys())[0:len(pipe)]
        

        for i in range(len(pipe)):
            # Mendapatkan parameter dari langkah 'sca'
            nama=first_step_name[i]
            sca_params = pipe.get_params()[nama]

                # Membuka run MLflow
            with mlflow.start_run( experiment_id=experiment_id,run_id=run_id) as run:

                # Log parameter secara otomatis menggunakan loop
                for param_name, param_value in sca_params.get_params().items():
                    param_name=nama+'__'+param_name
                    #print(param_name,param_value)
                    mlflow.log_param(param_name, param_value)
                    
                    
                #====================metric===================
                #matric
                hasil_test=grid_search.predict(X_test)
 
                
                
                
                                # Menghitung akurasi
                accuracy = accuracy_score(y_test, hasil_test)

                # Menghitung F1 score
                f1 = f1_score(y_test, hasil_test,average='macro')

                # Menghitung recall
                recall = recall_score(y_test, hasil_test,average='macro')

                
                
                                # Menghitung precision
                precision = precision_score(y_test, hasil_test,average='macro')


   
                

                # log model 
                mlflow.sklearn.log_model(sk_model=grid_search, artifact_path="grid_search__"+str(increment_number-1),signature=model_signature)

                metrics = {
                    'mean_test_score':pd.DataFrame(grid_search.cv_results_)['mean_test_score'].values[0],
                    "accuracy": accuracy,
                    "f1": f1,
                    "recall": recall,
                    "precision": precision,
         
                }

                mlflow.log_metrics(metrics)
                
                
        mlflow.end_run()
    print('selesai')

        
    return increment_number


     

def ambil_best(grid_search,n):
    cv_r=grid_search.copy()
    cv_r_v=cv_r.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
    cv_r_v=cv_r_v.head(n)['params'].values

    for wi in range(len(cv_r_v)):

        for i in list(cv_r_v[wi]):

            cv_r_v[wi][i]=[cv_r_v[wi][i]]


    return cv_r_v


In [2]:
import mlflow.sklearn
from sklearn.metrics import classification_report, confusion_matrix


def test(model_paths,X_test2,y_test):
    ii=1
    for i, model_path in enumerate(model_paths, start=1):
        print(f'\n\n================================| model {ii} |==========================================')
        
        model = mlflow.sklearn.load_model(model_path)
        y_pred = model.predict(X_test2)
        print(f"Classification Report for model {model_path[-2:]}:")
        print(classification_report(y_test, y_pred))
        print(f"Confusion Matrix for model {model_path[-2:]}:\n", confusion_matrix(y_test, y_pred), '\n\n')
        
        ii=ii+1

## Imports

In [3]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats



import seaborn as sns
import statsmodels.api as sm
import pandas as pd



from scipy.stats import shapiro,normaltest,kstest,jarque_bera
import pingouin as pg
from pingouin import kruskal


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import mlflow


from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score


import mlflow

from mlflow.models.signature import ModelSignature
from mlflow.models.signature import infer_signature
from mlflow.types.schema import Schema
from mlflow.types.schema import ParamSchema
from mlflow.types.schema import ParamSpec
from mlflow.types.schema import ColSpec
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import pandas as pd
from typing import Tuple
from sklearn import set_config
set_config(display='diagram')
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression


In [4]:
df = pd.read_csv('data/data_csv/data_bersih.csv')

In [5]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,virginica
145,6.3,2.5,5.0,1.9,virginica
146,6.5,3.0,5.2,2.0,virginica
147,6.2,3.4,5.4,2.3,virginica


In [6]:
X = df.drop('species',axis=1)
y = df['species']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=141)

In [8]:
y_train.value_counts()

setosa        36
versicolor    35
virginica     33
Name: species, dtype: int64

In [9]:
y_test.value_counts()

virginica     17
versicolor    15
setosa        13
Name: species, dtype: int64

In [10]:
from sklearn.feature_selection import mutual_info_classif
mutual_info=mutual_info_classif(X_train, y_train)

mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending=False)

petal_length    1.025486
petal_width     0.965425
sepal_length    0.491139
sepal_width     0.221138
dtype: float64

In [11]:

scatler = StandardScaler()
log_model = LogisticRegression(max_iter=10000)

In [12]:
opera=[('StandardScaler',scatler),('LogisticRegression',log_model)]
pipeline=Pipeline(opera)

## Logistic Regression Model  try all

In [13]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

pipeline

Pipeline(steps=[('StandardScaler', StandardScaler()),
                ('LogisticRegression', LogisticRegression(max_iter=10000))])

## param_grid_lbfgs

In [14]:
param_grid_lbfgs = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['lbfgs'],
    'LogisticRegression__penalty': ['l2', 'none']  # Perbaikan penulisan untuk 'none'
}


In [15]:
grid_search = GridSearchCV(pipeline, param_grid_lbfgs , cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'none'],
                         'LogisticRegression__solver': ['lbfgs'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [16]:
df_1 = pd.DataFrame(grid_search.cv_results_)
df_1.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_1 = df_1.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_1.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.021082,0.007735,0.000698,0.001396,2.50075,l2,lbfgs,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
21,0.008671,0.005670,0.003086,0.005502,5.0005,l2,lbfgs,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.952381,0.952381,0.904762,0.95,0.951905,0.030132,5
6,0.012568,0.007374,0.004557,0.006201,0.001,none,lbfgs,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",1.000000,0.904762,0.952381,0.904762,0.95,0.942381,0.035520,8
5,0.018582,0.008062,0.000638,0.001277,0.001,none,lbfgs,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.90,0.932381,0.024541,14
7,0.013714,0.003073,0.004329,0.003880,0.001,none,lbfgs,5.0005,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.904762,0.952381,0.809524,0.857143,0.90,0.884762,0.048225,19
12,0.016124,0.003948,0.000523,0.001045,2.50075,l2,lbfgs,5.0005,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.904762,0.809524,0.857143,0.90,0.875238,0.037423,27
8,0.019815,0.006220,0.002039,0.002057,0.001,none,lbfgs,7.50025,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.809524,0.857143,0.80,0.817143,0.036341,28
1,0.012885,0.004996,0.001611,0.003222,0.001,l2,lbfgs,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.857143,0.761905,0.80,0.807619,0.042762,37
9,0.012679,0.002799,0.002838,0.002382,0.001,none,lbfgs,10.0,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.809524,0.761905,0.80,0.798095,0.035328,41
0,0.017864,0.004416,0.003129,0.006258,0.001,l2,lbfgs,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.714286,0.666667,0.619048,0.761905,0.85,0.722381,0.079619,50


## param_grid_newton-cg 

In [17]:
param_grid_newton = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['newton-cg'],
    'LogisticRegression__penalty': ['l2', 'none']  # Perbaikan penulisan untuk 'none'
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_newton , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'none'],
                         'LogisticRegression__solver': ['newton-cg'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [18]:
df_2 = pd.DataFrame(grid_search.cv_results_)
df_2.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_2 = df_2.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_2.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,0.015382,0.004203,0.004916,0.004756,0.001,none,newton-cg,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
11,0.011036,0.004404,0.003953,0.005387,2.50075,l2,newton-cg,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.904762,0.952381,0.952381,0.95,0.951905,0.030132,12
5,0.047504,0.011684,0.002943,0.003690,0.001,none,newton-cg,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.90,0.932381,0.024541,14
7,0.013720,0.002255,0.002316,0.001513,0.001,none,newton-cg,5.0005,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.809524,0.904762,0.904762,0.857143,0.95,0.885238,0.047913,19
8,0.012760,0.004750,0.000849,0.001062,0.001,none,newton-cg,7.50025,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.809524,0.857143,0.904762,0.857143,0.80,0.845714,0.037808,28
1,0.009116,0.006311,0.006045,0.005976,0.001,l2,newton-cg,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.857143,0.761905,0.80,0.807619,0.042762,37
0,0.016842,0.002407,0.001299,0.001676,0.001,l2,newton-cg,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.714286,0.666667,0.619048,0.761905,0.85,0.722381,0.079619,50


## param_grid_liblinear

In [19]:
param_grid_liblinear = {
    'LogisticRegression__tol': np.linspace(0.001, 10,5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['liblinear'],
    'LogisticRegression__penalty': ['l2', 'l1']  
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_liblinear , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'l1'],
                         'LogisticRegression__solver': ['liblinear'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [20]:
df_3 = pd.DataFrame(grid_search.cv_results_)
df_3.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_3 = df_3.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_3.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
25,0.005711,0.003083,0.002219,0.001156,5.0005,l1,liblinear,0.001,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.904762,0.904762,0.952381,0.95,0.942381,0.035520,1
15,0.007284,0.001809,0.002757,0.001923,2.50075,l1,liblinear,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.904762,0.904762,0.904762,0.95,0.932857,0.037868,4
40,0.006524,0.004424,0.005904,0.005421,10.0,l2,liblinear,0.001,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.952381,0.904762,0.904762,0.904762,0.95,0.923333,0.022758,5
20,0.006037,0.004268,0.005442,0.001851,5.0005,l2,liblinear,0.001,"{'LogisticRegression__C': 5.000500000000001, '...",0.952381,0.904762,0.904762,0.857143,0.95,0.913810,0.035135,6
10,0.007775,0.001377,0.001469,0.001344,2.50075,l2,liblinear,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.904762,0.904762,0.857143,0.95,0.904286,0.029370,8
36,0.010306,0.004043,0.001882,0.003765,7.50025,l1,liblinear,2.50075,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.857143,0.904762,0.809524,0.761905,0.85,0.836667,0.048083,9
26,0.004821,0.000982,0.001988,0.001200,5.0005,l1,liblinear,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",0.857143,0.857143,0.761905,0.809524,0.80,0.817143,0.036341,10
0,0.010971,0.002832,0.003160,0.003216,0.001,l2,liblinear,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.761905,0.857143,0.857143,0.761905,0.80,0.807619,0.042762,11
11,0.008678,0.001844,0.001723,0.002241,2.50075,l2,liblinear,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.761905,0.857143,0.761905,0.761905,0.80,0.788571,0.037326,13
16,0.008560,0.006236,0.001687,0.001651,2.50075,l1,liblinear,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.809524,0.857143,0.761905,0.714286,0.80,0.788571,0.047961,13


## param_grid_sag

In [21]:

param_grid_sag = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['sag'],
    'LogisticRegression__penalty': ['l2', 'none']  
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_sag , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l2', 'none'],
                         'LogisticRegression__solver': ['sag'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [22]:
df_4 = pd.DataFrame(grid_search.cv_results_)
df_4.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_4 = df_4.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_4.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.002608,0.002449,0.002323,0.002883,2.50075,l2,sag,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
5,0.020655,0.007204,0.004101,0.004061,0.001,none,sag,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.95,0.942381,0.018832,5
19,0.007223,0.006786,0.001474,0.001710,2.50075,none,sag,10.0,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.857143,0.952381,0.952381,0.95,0.942381,0.046569,5
26,0.007626,0.005848,0.002684,0.003009,5.0005,none,sag,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",0.904762,0.904762,0.904762,0.857143,0.90,0.894286,0.018663,11
38,0.009145,0.003679,0.000459,0.000918,7.50025,none,sag,7.50025,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.904762,0.857143,1.000000,0.809524,0.90,0.894286,0.063059,11
39,0.007554,0.004751,0.004678,0.003307,7.50025,none,sag,10.0,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.952381,0.904762,0.857143,0.761905,0.95,0.885238,0.070829,13
34,0.006040,0.002281,0.002180,0.002077,7.50025,l2,sag,10.0,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.952381,0.904762,0.666667,0.952381,0.95,0.885238,0.110784,13
41,0.008145,0.005835,0.005170,0.006565,10.0,l2,sag,2.50075,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.952381,0.857143,0.904762,0.809524,0.90,0.884762,0.048225,15
14,0.006467,0.000764,0.003024,0.001157,2.50075,l2,sag,10.0,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.952381,0.857143,0.952381,0.761905,0.90,0.884762,0.071040,15
43,0.007695,0.005205,0.005416,0.004444,10.0,l2,sag,7.50025,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.857143,0.952381,0.904762,0.857143,0.85,0.884286,0.039256,17


## param_grid_saga

In [23]:
param_grid_saga = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['saga'],
    'LogisticRegression__penalty': ['l1','l2', 'none'] 
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_saga , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__penalty': ['l1', 'l2', 'none'],
                         'LogisticRegression__solver': ['saga'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [24]:
df_5 = pd.DataFrame(grid_search.cv_results_)
df_5.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_5 = df_5.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_5.head(10)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
15,0.016591,0.002506,0.004470,0.004495,2.50075,l1,saga,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
10,0.013955,0.003215,0.004801,0.006188,0.001,none,saga,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.95,0.942381,0.018832,9
14,0.007347,0.004635,0.000592,0.001184,0.001,none,saga,10.0,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.904762,1.000000,0.857143,0.95,0.932857,0.048384,14
48,0.008059,0.001669,0.003037,0.001078,7.50025,l1,saga,7.50025,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.904762,0.857143,1.000000,0.904762,0.95,0.923333,0.048291,15
54,0.007600,0.007028,0.001875,0.002557,7.50025,l2,saga,10.0,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.904762,0.904762,0.904762,0.904762,0.95,0.913810,0.018095,16
32,0.005895,0.007261,0.000492,0.000983,5.0005,l1,saga,5.0005,"{'LogisticRegression__C': 5.000500000000001, '...",0.952381,0.904762,0.952381,0.809524,0.95,0.913810,0.055213,16
64,0.009324,0.003528,0.001859,0.001885,10.0,l1,saga,10.0,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",0.904762,0.904762,0.952381,0.904762,0.90,0.913333,0.019611,18
13,0.007757,0.006993,0.001241,0.002482,0.001,none,saga,7.50025,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.904762,1.000000,0.857143,0.85,0.912857,0.057055,19
18,0.011852,0.004656,0.000779,0.001121,2.50075,l1,saga,7.50025,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.857143,0.952381,0.857143,0.95,0.904286,0.042067,20
22,0.007707,0.006615,0.004332,0.004266,2.50075,l2,saga,5.0005,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.952381,0.952381,0.857143,0.809524,0.95,0.904286,0.059864,20


In [25]:
param_grid_saga2 = {
    'LogisticRegression__tol': np.linspace(0.001, 10, 5),
    'LogisticRegression__C': np.linspace(0.001, 10, 5),
    'LogisticRegression__solver': ['saga'],
    'LogisticRegression__l1_ratio': np.linspace(0.001, 10, 1),
    'LogisticRegression__penalty': ['elasticnet']  
}

# Inisialisasi GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid_saga2 , cv=5, scoring='accuracy')

# Lakukan pencarian grid
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('StandardScaler', StandardScaler()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=10000))]),
             param_grid={'LogisticRegression__C': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01]),
                         'LogisticRegression__l1_ratio': array([0.001]),
                         'LogisticRegression__penalty': ['elasticnet'],
                         'LogisticRegression__solver': ['saga'],
                         'LogisticRegression__tol': array([1.00000e-03, 2.50075e+00, 5.00050e+00, 7.50025e+00, 1.00000e+01])},
             scoring='accuracy')

In [26]:
df_6 = pd.DataFrame(grid_search.cv_results_)
df_6.drop_duplicates(subset=['mean_test_score', 'std_test_score', 'rank_test_score'], inplace=True)
df_6 = df_6.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])
df_6.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__l1_ratio,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,0.009987,0.006627,0.005786,0.006698,2.50075,0.001,elasticnet,saga,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1
19,0.011206,0.006823,0.002527,0.005054,7.50025,0.001,elasticnet,saga,10.0,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",1.000000,0.904762,0.904762,0.857143,0.90,0.913333,0.046890,5
9,0.006902,0.003788,0.003823,0.003765,2.50075,0.001,elasticnet,saga,10.0,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",0.904762,0.904762,1.000000,0.904762,0.85,0.912857,0.048459,6
11,0.007463,0.003009,0.005716,0.002092,5.0005,0.001,elasticnet,saga,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",0.952381,0.857143,0.857143,0.904762,0.90,0.894286,0.035431,7
14,0.006853,0.007364,0.003136,0.006272,5.0005,0.001,elasticnet,saga,10.0,"{'LogisticRegression__C': 5.000500000000001, '...",0.904762,0.952381,0.809524,0.904762,0.90,0.894286,0.046501,7
23,0.006800,0.006709,0.003543,0.006105,10.0,0.001,elasticnet,saga,7.50025,"{'LogisticRegression__C': 10.0, 'LogisticRegre...",1.000000,0.904762,0.857143,0.809524,0.90,0.894286,0.063059,7
13,0.005851,0.005973,0.006080,0.005536,5.0005,0.001,elasticnet,saga,7.50025,"{'LogisticRegression__C': 5.000500000000001, '...",0.952381,0.904762,0.904762,0.857143,0.85,0.893810,0.037240,10
18,0.008472,0.004386,0.003006,0.003370,7.50025,0.001,elasticnet,saga,7.50025,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.857143,0.952381,0.952381,0.761905,0.90,0.884762,0.071040,11
12,0.011383,0.004403,0.001756,0.001856,5.0005,0.001,elasticnet,saga,5.0005,"{'LogisticRegression__C': 5.000500000000001, '...",0.857143,0.904762,1.000000,0.809524,0.85,0.884286,0.065285,12
16,0.011187,0.004754,0.002443,0.003110,7.50025,0.001,elasticnet,saga,2.50075,"{'LogisticRegression__C': 7.50025, 'LogisticRe...",0.952381,0.904762,0.952381,0.809524,0.80,0.883810,0.066911,13


# gabung kan semua ambilyg top 5

In [27]:
df_gabungan=pd.concat([df_1.head(),df_2.head(),df_3.head(),df_4.head(),df_5.head(),df_6.head()])

df_gabungan.drop_duplicates(subset=['mean_test_score', 'std_test_score'], inplace=True)
df_gabungan.sort_values(by=['mean_test_score', 'std_test_score'], ascending=[False, True])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogisticRegression__C,param_LogisticRegression__penalty,param_LogisticRegression__solver,param_LogisticRegression__tol,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,param_LogisticRegression__l1_ratio
10,0.021082,0.007735,0.000698,0.001396,2.50075,l2,lbfgs,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.952381,0.952381,0.952381,1.00,0.971429,0.023328,1,NaN
21,0.008671,0.005670,0.003086,0.005502,5.0005,l2,lbfgs,2.50075,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.952381,0.952381,0.904762,0.95,0.951905,0.030132,5,NaN
11,0.011036,0.004404,0.003953,0.005387,2.50075,l2,newton-cg,2.50075,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.904762,0.952381,0.952381,0.95,0.951905,0.030132,12,NaN
5,0.020655,0.007204,0.004101,0.004061,0.001,none,sag,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.95,0.942381,0.018832,5,NaN
25,0.005711,0.003083,0.002219,0.001156,5.0005,l1,liblinear,0.001,"{'LogisticRegression__C': 5.000500000000001, '...",1.000000,0.904762,0.904762,0.952381,0.95,0.942381,0.035520,1,NaN
19,0.007223,0.006786,0.001474,0.001710,2.50075,none,sag,10.0,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.857143,0.952381,0.952381,0.95,0.942381,0.046569,5,NaN
6,0.012568,0.007374,0.004557,0.006201,0.001,none,lbfgs,2.50075,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",1.000000,0.904762,0.952381,0.904762,0.95,0.942381,0.035520,8,NaN
14,0.007347,0.004635,0.000592,0.001184,0.001,none,saga,10.0,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.904762,1.000000,0.857143,0.95,0.932857,0.048384,14,NaN
15,0.007284,0.001809,0.002757,0.001923,2.50075,l1,liblinear,0.001,"{'LogisticRegression__C': 2.50075, 'LogisticRe...",1.000000,0.904762,0.904762,0.904762,0.95,0.932857,0.037868,4,NaN
5,0.018582,0.008062,0.000638,0.001277,0.001,none,lbfgs,0.001,"{'LogisticRegression__C': 0.001, 'LogisticRegr...",0.952381,0.952381,0.904762,0.952381,0.90,0.932381,0.024541,14,NaN


In [29]:
data= ambil_best(df_gabungan,10)
data

array([{'LogisticRegression__C': [2.50075], 'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['lbfgs'], 'LogisticRegression__tol': [0.001]},
       {'LogisticRegression__C': [5.000500000000001], 'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['lbfgs'], 'LogisticRegression__tol': [2.50075]},
       {'LogisticRegression__C': [2.50075], 'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['newton-cg'], 'LogisticRegression__tol': [2.50075]},
       {'LogisticRegression__C': [0.001], 'LogisticRegression__penalty': ['none'], 'LogisticRegression__solver': ['sag'], 'LogisticRegression__tol': [0.001]},
       {'LogisticRegression__C': [5.000500000000001], 'LogisticRegression__penalty': ['l1'], 'LogisticRegression__solver': ['liblinear'], 'LogisticRegression__tol': [0.001]},
       {'LogisticRegression__C': [2.50075], 'LogisticRegression__penalty': ['none'], 'LogisticRegression__solver': ['sag'], 'LogisticRegression__tol': [10.0]},
     

In [30]:
if __name__ == "__main__":

    experiment_id = mlflow.create_experiment(
        name="project_logistic_iris_new",
        tags={"env": "dev", "version": "1.0.0"},
    )

    print(experiment_id)
    

mlflow.end_run()


786086141322766022


In [31]:
run_name = "log___"
increment_number = 1

In [32]:

increment_number=input_mlflow(data,increment_number,X_train, y_train,pipeline,run_name,experiment_id,X_test,y_test)

mlruns/786086141322766022/5f09fd1bafb448a2aef267f5fadaba46/artifacts/grid_search__1
mlruns/786086141322766022/830d94ec8ed1499ba069ff6e1bcb5c3e/artifacts/grid_search__2
mlruns/786086141322766022/bfcad8f79a9e4ca1b949454c0de543c1/artifacts/grid_search__3
mlruns/786086141322766022/11945cb68bfa4d1980271db5cbd2db1c/artifacts/grid_search__4
mlruns/786086141322766022/b3803ca1c68d417bacfa0bee453d7ea8/artifacts/grid_search__5
mlruns/786086141322766022/cc1c5671ba3a471984265424df1d981a/artifacts/grid_search__6
mlruns/786086141322766022/7cf10f11aee44fd69c3e10489b3d25e2/artifacts/grid_search__7
mlruns/786086141322766022/347a0bc37642476f86a0b977a3a0894c/artifacts/grid_search__8
mlruns/786086141322766022/aefba391402044a2965aad8b0ae1483f/artifacts/grid_search__9
mlruns/786086141322766022/57911d9e035a4ef4a695ac1ee42bded3/artifacts/grid_search__10
selesai


In [33]:
model_paths = [
  
    'mlruns/786086141322766022/5f09fd1bafb448a2aef267f5fadaba46/artifacts/grid_search__1',

    'mlruns/786086141322766022/830d94ec8ed1499ba069ff6e1bcb5c3e/artifacts/grid_search__2',
    'mlruns/786086141322766022/bfcad8f79a9e4ca1b949454c0de543c1/artifacts/grid_search__3'
        ]

test(model_paths,X_test,y_test)



================================| model 1 |==========================================
Classification Report for model _1:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      0.93      0.97        15
   virginica       0.94      1.00      0.97        17

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45

Confusion Matrix for model _1:
 [[13  0  0]
 [ 0 14  1]
 [ 0  0 17]] 




================================| model 2 |==========================================
Classification Report for model _2:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        13
  versicolor       1.00      0.93      0.97        15
   virginica       0.94      1.00      0.97        17

    accuracy                           0.98        45
   macro avg       0.98    